In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import SingleDayTradingStrategy as sdts

In [ ]:
def load_data(file_name):
    df = pd.read_csv(file_name)
    df.columns = [col.strip().lower() for col in df.columns]  # 标准化列名
    df['date'] = pd.to_datetime(df['date'])
    df['datetime'] = df['date'] + pd.to_timedelta(df['minute'], unit='m')
    df.set_index('datetime', inplace=True)
    df = df.sort_index()
    return df

In [ ]:
def plot_result(equity_df, daily_summary):
    fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=False)
    equity_df['equity'].plot(ax=axes[0], title='Equity Curve')
    axes[0].set_ylabel('Equity ($)')
    daily_summary['daily_pnl'].plot(kind='bar', ax=axes[1], title='Daily PnL', color='orange')
    axes[1].set_ylabel('PnL ($)')
    plt.tight_layout()
    plt.show()

In [ ]:
def run_intraday_highlow_strategy(df, startDate, days=7, initial_cash=100_000, window=5, plot=True):

    grouped = df.groupby('date')
    cash = initial_cash

    all_trades = []
    all_exception_bars = []
    equity_curve = []

    count = 0
    for date, day in grouped:
        if startDate and date < startDate:
            continue
        
        count += 1
        if count > days:
            break
        trades, day_equity_curve, cash, exception_bars = sdts.SingleDayTradingStrategy(date, cash, window).run_single_day(day.copy())
        all_trades.extend(trades)
        equity_curve.extend(day_equity_curve)
        all_exception_bars.extend(exception_bars)

    if len(all_trades) == 0:
        print("No trades executed in the given period.")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    trades_df = pd.DataFrame(all_trades)
    equity_df = pd.DataFrame(equity_curve).set_index('time')
    exception_bars_df = pd.DataFrame(all_exception_bars)

    daily_summary = trades_df.groupby('date').agg(
        daily_pnl=('pnl', 'sum'),
        trades=('pnl', 'count'),
        win_rate=('pnl', lambda x: (x > 0).mean())
    )

    if plot:
        plot_result(equity_df, daily_summary)

    return trades_df, equity_df, daily_summary, exception_bars_df

In [ ]:
# === 加载数据 ===
year = 2025
df = load_data(f"data/spy_{year}.csv")
startDate = pd.to_datetime(f"{year}-01-01")  # Convert to date object
days = 500
initial_cash = 1000
window = 5
plot = False
trades_df, equity_df, daily_summary, exception_bars = run_intraday_highlow_strategy(df, startDate=startDate, days=days, initial_cash=initial_cash, window=window, plot=plot)

In [ ]:
print(f"Total trades executed: {trades_df['date'].count()} for {daily_summary['daily_pnl'].count()} days starting from {startDate.date()}")

open_price = trades_df.iloc[0]['entry_price']
close_price = trades_df.iloc[-1]['exit_price']
market_win_rate = (close_price / open_price if open_price != 0 else 0) - 1
print(f"Open Price: {open_price:.2f}, Close Price: {close_price:.2f}")
print(f"Market Win Rate: {market_win_rate:.2%}")

open_equity = equity_df.iloc[0]['equity']
close_equity = equity_df.iloc[-1]['equity']
strategy_win_rate = (close_equity - open_equity) / open_price
print(f"Open Equity: {open_equity:.2f}, Close Equity: {close_equity:.2f}")
print(f"Strategy Win Rate: {strategy_win_rate:.2%}")

# print("equity_df:")
# print(equity_df.head())
# print(equity_df.tail())
print("\ntrades_df:")
print(trades_df.head())
print(trades_df.tail())
print("\ndaily_summary:")
print(daily_summary.head())
print(daily_summary.tail())
print("\nException Bars:")
print(exception_bars.head())